# PyGrid: A Peer-to-Peer Platform for Private Data Science and Federated Learning

## Using a simple example, this notebook introduces to beginners what PyGrid is and how you can use it for your data science projects. 

### Credits:
- Original authors: 
 - Emma Bluemke - Twitter: [@emmabluemke](https://twitter.com/emmabluemke) - Github: [@em-blue](@https://github.com/em-blue)
 - Ionesio Junior - Github: [@IonesioJunio](https://github.com/IonesioJunior)


- Reviewers: 
 - Patrick Cason - Github: [@cereallcerny](https://github.com/cereallarceny)
 - Juan M. Aunon - Twitter: [@jm_aunon](https://twitter.com/jm_aunon) - Github: [@jmaunon](https://github.com/jmaunon)


- New Content tested and enriched by: 
 - Dev Bharti - Github: [@devtastic](https://github.com/devtastic)
 - Nahua Kang - Github: [@nahuakang](https://github.com/nahuakang)
 
 
 _This notebook covers the first part of [what-is-pygrid-demo](https://blog.openmined.org/what-is-pygrid-demo/)_

![](images/1.png)

## __What if you could train on all of the world’s data without that data leaving its device while keeping that data private?__

PyGrid is a peer-to-peer platform for private data science and federated learning. With PyGrid, data owners can provide, monitor, and __manage access to their own private data clusters. The data does not leave the data owner’s server__.

Data scientists and researchers can then use PyGrid to perform __privacy-preserving statistical analysis__ on the private dataset, or even perform __federated learning across multiple institution’s datasets__.

### This blog post will cover:

1. Basic concepts such as __Federated Learning__ and __Secure Multi-party Computation__, which are necessary for understanding __PyGrid__.
2. The __PySyft__ library and the __PyGrid__ platform. PySyft is a privacy-preserving machine learning library that is deployed using the PyGrid platform.
3. __Practical examples__ of privacy-preserving analysis using PyGrid, which offer a peek into PyGrid’s architecture and illustrate how you can apply PyGrid to real problems.

# Federated Learning

Before embarking on our journey in PyGrid, w should understand the basics of Federated Learning. Federated Learning is a machine learning technique that trains AI models on data hosted across multiple decentralized servers and edge devices without exchanging the data. How does this work?

First, the data scientist create an initial model. Then, they would send this model to a dataset owner - in this case, Joe’s device (see illustration below).

![](images/4.png)

Now, Joe can update this model by training it on his dataset. After training completes, Joe returns the updated model, as in the new model weights, to the data scientist.

![](images/5.png)

After training on Joe's data, the data scientist decides to send the updated AI model to, say, Jane’s device. Therefore, Jane updates the data scientist's model by training it on her dataset. After training on her data completes, Jane sends the updated weights back to the data scientist.

![](images/6.png)

The data scientist's model has learned from both Joe and Jane’s data. But the story does not end here. In federated learning, the data scientist can repeat this process over many nodes, or even train models simultaneously on multiple nodes and average the updated weights, allowing for faster improvements to the model.

The main benefits of federated learning are:

- Training data stays on the host servers and edge devices (for example, the hospital servers or a user's smartphone)
- Privacy-focused solution that enhances protection for sensitive data
- Reduce legal liability for model owners (i.e. the data scientists or companies)
- Reduce network bandwidth involved in uploading large datasets for training

There are many potential use cases for federated learning. For example,

- [A scientist training on data from multiple hospitals](https://blog.openmined.org/federated-learning-differential-privacy-and-encrypted-computation-for-medical-imaging/)
- [A smartphone app training on data from multiple phones](https://blog.openmined.org/apheris-openmined-pytorch-announcement/)
- [A company leaning to predict data when it’s machines need maintenance by training models across data from many sensors](https://blog.openmined.org/predictive-maintenance-of-turbofan-engines-using-federated-learning/)

## __Secure Multi-Party Computation__

Secure Multi-Party Computation (SMPC) is a different way to encrypt data and then distribute the data to different devices. The main advantage is that, unlike traditional cryptography, SMPC keeps the encrypted data secret yet _alive_, thus allowing us to perform logical and arithmetic operations on the encrypted data.

How can you perform math with multi-party computation? Below is a (very) simplified example of how SMPC works:

![](images/2.png)

In this example (see illustration above), we have Andrew holding a piece of sensitive personal data, his special number 5. He can anonymize his data by decomposing the number 5 into two or more different numbers. Imagine that Andrew decides to decompose the number 5 into 2 and 3. This way, he can share his anonymized data with his friends Marianne and Bob.

Neither Marianne nor Bob knows the real value of Andrew’s data because they each holds only a part of the special number. And since the decomposed numbers are still _alive_, either of them can perform any kind of operation without seeking the agreement of the other party. Indeed, while these numbers are encrypted between them, we’ll still be able to perform computations. __That way, we can use encrypted values to compute user’s data without showing any kind of sensitive information__.

![](images/3.png)

With these two concepts, let's now move on to PySyft and PyGrid.

## __Use Case: Private Statistical Analysis__
Let’s explore the following scenario with two work flows:

1. A __data owner__ that wants to publish their sensitive data on their node. (In this case, a hospital pediatric ward).
2. A __data scientist__ that wants to find a specific dataset over the grid network to compute some statistical analysis.

## __The Data Owner__
### __Step 1: Import PySyft and dependencies__
To publish the sensitive yet valuable data, the data owner first imports PyGrid and PySyft dependencies.

In this example, we’ll import `syft` and wrap the standard `torch` modules with `syft.TorchHook` to add the additional privacy-preserving features that PySyft offers. If you have not set up your computer, follow [this](https://github.com/OpenMined/PyGrid) to install the necessary dependencies.

Also, please make sure [these](https://github.com/OpenMined/PyGrid/tree/dev/examples#running-the-examples) instructions have been followed.

Once done, you should have the following components:

 - PyGrid Network      (http://network:7000)
 - PyGrid Node Bob (http://bob:5001)
 - PyGrid Node Alice   (http://alice:5000)
 
This tutorial assumes that these components are running in background. See [instructions](https://github.com/OpenMined/PyGrid/tree/dev/examples#how-to-run-this-tutorial) for more details. 
Yours URL / ports might vary depending on your configuration, so please ensure you look at the output of `docker-compose up` before moving forward.

Visit http://localhost:7000/connected-nodes to ensure all nodes are ready and setup! You should get a response like below for the standard setup:

`{"grid-nodes": ["Dan", "Alice", "Charlie", "Bob"]}`

In [1]:
import syft as sy
from syft.grid.clients.data_centric_fl_client import DataCentricFLClient
from syft.grid.clients.model_centric_fl_client import ModelCentricFLClient

import torch as th
hook = sy.TorchHook(th)

In [2]:
th.cuda.is_available()

True

In [3]:
import jedi
jedi.__version__

'0.18.0'

In [5]:
import IPython
IPython.__version__

'7.19.0'

In [11]:
!pip install --use-deprecated=legacy-resolver IPython==r 

ERROR: Could not find a version that satisfies the requirement IPython==r (from versions: 0.10, 0.10.1, 0.10.2, 0.11, 0.12, 0.12.1, 0.13, 0.13.1, 0.13.2, 1.0.0, 1.1.0, 1.2.0, 1.2.1, 2.0.0, 2.1.0, 2.2.0, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 3.0.0, 3.1.0, 3.2.0, 3.2.1, 3.2.2, 3.2.3, 4.0.0b1, 4.0.0, 4.0.1, 4.0.2, 4.0.3, 4.1.0rc1, 4.1.0rc2, 4.1.0, 4.1.1, 4.1.2, 4.2.0, 4.2.1, 5.0.0b1, 5.0.0b2, 5.0.0b3, 5.0.0b4, 5.0.0rc1, 5.0.0, 5.1.0, 5.2.0, 5.2.1, 5.2.2, 5.3.0, 5.4.0, 5.4.1, 5.5.0, 5.6.0, 5.7.0, 5.8.0, 5.9.0, 5.10.0, 6.0.0rc1, 6.0.0, 6.1.0, 6.2.0, 6.2.1, 6.3.0, 6.3.1, 6.4.0, 6.5.0, 7.0.0b1, 7.0.0rc1, 7.0.0, 7.0.1, 7.1.0, 7.1.1, 7.2.0, 7.3.0, 7.4.0, 7.5.0, 7.6.0, 7.6.1, 7.7.0, 7.8.0, 7.9.0, 7.10.0, 7.10.1, 7.10.2, 7.11.0, 7.11.1, 7.12.0, 7.13.0, 7.14.0, 7.15.0, 7.16.0, 7.16.1, 7.17.0, 7.18.0, 7.18.1, 7.19.0)
ERROR: No matching distribution found for IPython==r


In [13]:
!pip search IPython --list-all-versions


Usage:   
  pip search [options] <query>

no such option: --list-all-versions


## __Step 2: Connect your node__

The next step is to connect with your own node. It’s important to note that the node app was deployed in some environment and you need to know its address previously. In this case we’ll connect with the hospital node (Bob) on our `http://localhost:5001`.

In [3]:
grid_address = "http://network:7000"
bob = "http://bob:5001"

In [4]:
hospital_datacluster = DataCentricFLClient(hook, bob)
hospital_datacluster

<Federated Worker id:Bob>

## __Step 3: Prepare data as tensors and add brief description__

Now, let's prepare our dataset to be published on the hospital node. To provide a clear understanding about the data that we publish, we should add a brief description that explains the data and its schema. For this example, let's publish the hospital’s monthly birth records.

In [5]:
data_description = """Description:
                        This data presents the monthly birth records.
                        
                        Columns:
                            Gender: 0 - Male, 1 - Female
                            Weight: in Kg
                            Height: in cm


                        Shape 5 * 3
                        """

monthly_birth_records = th.tensor([[1, 3.5, 47.3],
                                   [0, 3.7, 48.1],
                                   [0, 3.9, 50.0],
                                   [1, 4.1, 52.3],
                                   [0, 4.1, 49.7]
                                  ])

## __Step 4: Define access rules and permissions__

After defining the data schema, we should define the rules that control data access. Here, we’re allowing some users (Bob, Ana and Alice) to get total access to the real values of this data.

In [6]:
private_dataset = monthly_birth_records.private_tensor(allowed_users = ("Bob", "Ana", "Alice"))

## __Step 5: Add tags and labels to help data scientists find your dataset__

Just like on your favorite social media platform, we need to add some tags to identify and label our data so that our data is more accessible from queries of data scientists who seek specific types of datasets.

In this example, we’re adding two tags: __#February__ to identify the month and __#birth-records__ to identify the data meaning.

In [7]:
private_dataset = private_dataset.tag("#February", "#birth-records").describe(data_description)
private_dataset

(Wrapper)>PrivateTensor>tensor([[ 1.0000,  3.5000, 47.3000],
        [ 0.0000,  3.7000, 48.1000],
        [ 0.0000,  3.9000, 50.0000],
        [ 1.0000,  4.1000, 52.3000],
        [ 0.0000,  4.1000, 49.7000]])

## __Step 6: Publish! You’re done.__

Now, the data is ready to be published. It’s important to note that you need to be allowed to publish private datasets on this node. In this example, we’re using Bob’s credential to publish this data on the node.

In [8]:
data_pointer = private_dataset.send(hospital_datacluster, user = "Bob")
data_pointer

(Wrapper)>[PointerTensor | me:29842312121 -> Bob:21160968527]
	Tags: #February #birth-records 
	Shape: torch.Size([5, 3])
	Description: Description:...

As the data owner, that's all that we need to do!

## __The Data Scientist__

In [9]:
from syft.grid.public_grid import PublicGridNetwork

grid = PublicGridNetwork(hook, grid_address)
results = grid.search("#February", "#birth-records")
results

{'Bob': [(Wrapper)>[PointerTensor | me:71313635971 -> Bob:21160968527]
  	Tags: #February #birth-records 
  	Shape: torch.Size([5, 3])
  	Description: Description:...]}

## __Step 1: Import PySyft and dependencies__

As a data scientist, we also need to import the `PySyft` library and wrap `torch` modules using `syft.TorchHook`.

In [10]:
import syft as sy
from syft.grid.public_grid import PublicGridNetwork

import torch as th
hook = sy.TorchHook(th)

## __Step 2: Connect to Grid Platform__

Unlike the data owner who published the dataset, the data scientist does not know where the nodes and the datasets are, so we first need to connect with the GridNetwork. The address of the grid network will be the gateway component address.

In [11]:
grid = PublicGridNetwork(hook, grid_address)
grid

## __Step 3: What data are you looking for? Search the network.__

After connecting with the grid network, we can search for the desired dataset tags. Perhaps you’re looking for x-rays of pneumonia, or hospital birth records. For this tutorial, we’re searching for the same tags that we’ve published before as the data owner. The grid network will return a dictionary containing the node’s ids as keys and data pointers as values.

In [12]:
results = grid.search("#February", "#birth-records")
results

{'Bob': [(Wrapper)>[PointerTensor | me:68464049766 -> Bob:21160968527]
  	Tags: #February #birth-records 
  	Shape: torch.Size([5, 3])
  	Description: Description:...]}

## __Step 4: Create a reference to that data pointer__

Next, we define a direct reference to the hospital’s data pointer.

In [13]:
feb_records = results['Bob'][0]
print(feb_records.description)

Description:
                        This data presents the monthly birth records.
                        
                        Columns:
                            Gender: 0 - Male, 1 - Female
                            Weight: in Kg
                            Height: in cm


                        Shape 5 * 3
                        


## __Wait - what if I try to copy that data?__

If we try to retrieve the real values of the data pointer without being allowed, an exception will be raised. This is how PyGrid can keep the data on the owner’s hands and empower them with the control to allow or deny access to the data samples.

In [14]:
import traceback
try:
    feb_records.get()
except Exception: # GetNotPermittedError
    traceback.print_exc()

Traceback (most recent call last):
  File "<ipython-input-14-6128d61299fc>", line 3, in <module>
    feb_records.get()
  File "/usr/local/lib/python3.7/dist-packages/syft/frameworks/torch/tensors/interpreters/native.py", line 648, in get
    tensor = self.child.get(*args, user=user, reason=reason, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/syft/generic/pointers/pointer_tensor.py", line 326, in get
    tensor = ObjectPointer.get(self, user=user, reason=reason, deregister_ptr=deregister_ptr)
  File "/usr/local/lib/python3.7/dist-packages/syft/generic/pointers/object_pointer.py", line 269, in get
    obj = self.owner.request_obj(self.id_at_location, self.location, user, reason)
  File "/usr/local/lib/python3.7/dist-packages/syft/workers/base.py", line 605, in request_obj
    obj = self.send_msg(ObjectRequestMessage(obj_id, user, reason), location)
  File "/usr/local/lib/python3.7/dist-packages/syft/workers/base.py", line 319, in send_msg
    response = sy.serde.deserialize(b

## __Step 5: Perform your computations__

Even without copying the data, we are still able to perform remote computations on this data. In this example, we want to compute the average of the babies' weight and height. To do this, we need to sum the column values remotely.

In [15]:
def sum_column(dataset, column):
    sum_result = dataset[0][column].copy()
    for i in range(1, dataset.shape[0]):
        sum_result += dataset[i][column]
    return sum_result

Now, we can compute the weight sum remotely. It will generate other remote tensor.

In [16]:
weight_sum = sum_column(feb_records, 1)
weight_sum

(Wrapper)>[PointerTensor | me:88702312983 -> Bob:62658804114]

We can do the same thing with the height column.

In [17]:
height_sum = sum_column(feb_records, 2)
height_sum

(Wrapper)>[PointerTensor | me:27407243430 -> Bob:91299325293]

Now, we just need to retrieve the aggregated value using our credentials and divide the value by 5  which is the dataset size.

In [18]:
avg_weight = weight_sum.get(user='Bob')/5
avg_weight

(Wrapper)>PrivateTensor>tensor(3.8600)

We can do the same thing to get the height average. That way, we are able to compute the average weight and height  of the babies that was born on this month without get access to any sensitive data.

In [19]:
avg_height = height_sum.get(user='Bob')/5
avg_height

(Wrapper)>PrivateTensor>tensor(49.4800)

__Done! We know the average height and weight of the babies born in February without ever moving the dataset to our own server, and we never needed to receive any private information about the individual babies.__

# Congratulations!!! - Time to Join the Community!

Congratulations on completing this notebook tutorial! If you enjoyed this and would like to join the movement toward privacy preserving, decentralized ownership of AI and the AI supply chain (data), you can do so in the following ways!

### Star PyGrid on GitHub

The easiest way to help our community is just by starring the GitHub repos! This helps raise awareness of the cool tools we're building.

- [Star PyGrid](https://github.com/OpenMined/PyGrid)

### Join our Slack!

The best way to keep up to date on the latest advancements is to join our community! You can do so by filling out the form at [http://slack.openmined.org](http://slack.openmined.org)

### Join a Code Project!

The best way to contribute to our community is to become a code contributor! At any time you can go to PySyft GitHub Issues page and filter for "Projects". This will show you all the top level Tickets giving an overview of what projects you can join! If you don't want to join a project, but you would like to do a bit of coding, you can also look for more "one off" mini-projects by searching for GitHub issues marked "good first issue".

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PyGrid/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Donate

If you don't have time to contribute to our codebase, but would still like to lend support, you can also become a Backer on our Open Collective. All donations go toward our web hosting and other community expenses such as hackathons and meetups!

[OpenMined's Open Collective Page](https://opencollective.com/openmined)